# InfoGroup data

> Process and prepare InfoGroup dataset.

- toc: True

## Processing

Starting from original CSV files.

- Convert to unicode
- Validate against JSON schema. A few erroneous data entries are erased here (e.g. text in numerical column). Existing implementation uses datapackage validator and takes several days with single core.
- Save to disk in parquet format.
- Provide interface to load single year of data. Allow filtering, column selection and small (optionally random) sample.

In [ ]:
from nbdev import *
%nbdev_default_export infogroup

In [ ]:
%nbdev_export
import sys
import logging
import json
import gzip
import shutil
import multiprocessing as mp
import functools

import numpy as np
import pandas as pd
import fastparquet
from IPython import display
from joblib import Memory

from rurec import resources
from rurec.resources import Resource
from rurec import util

memory = Memory(resources.paths['root'] / 'cache')
data_years = range(1997, 2018)

In [ ]:
%nbdev_export
# for batch runs: log to a file
logging.basicConfig(filename=resources.paths['root'] / 'logs/processing.log', 
                    filemode='w', level=logging.INFO, format='%(asctime)s %(levelname)s:\n%(message)s', force=True)

In [ ]:
# for interactive use: log to stdout
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(asctime)s %(levelname)s:\n%(message)s', force=True)

In [ ]:
resources.add(Resource('infogroup/schema', '/InfoGroup/data/processed/schema.json', 'Processed InfoGroup data, schema', False))
resources.add(Resource('infogroup/full', '/InfoGroup/data/processed/full.pq', 'Processed InfoGroup data, all years, partitioned parquet', False))
for y in data_years:
    resources.add(Resource(f'infogroup/orig/{y}', f'/InfoGroup/data/original/raw/{y}_Business_Academic_QCQ.csv', f'Original unprocessed InfoGroup data, {y}', False))

In [ ]:
# override resource paths for testing and debugging
resources.get('infogroup/full').path = resources.paths['root'] / 'tmp/full.pq'

In [ ]:
# faster returns for testing and debugging
pd.read_csv = functools.partial(pd.read_csv, nrows=10_000)

## Clear and validate raw data

- Change "latin-1" encoding to "utf-8", remove double quotes around values.
- Remove double quotes around every value.
- Rename columns to ALL_CAPS.
- In 2009: pad string fields with zeroes.
- Validate values format, replace errors with missing values.

### Future work

- Correct 2-digit state part of the FIPS code.
- Correct missing CBSA code and CBSA level, mainly in 2009.
- add indicator variables for different samples (random, WI, FAI, ...) to be used as parquet partitions to allow quick read of data subsets
- put meaningfult labels to categoricals ("1-5" instead if "A" etc).
  - This will be tricky for POPULATION_CODE that changes coding between 2015 and 2016
- make up and add enum constraints for TITLE_CODE and CALL_STATUS_CODE
- add logging of errors to a file
- if categoricals are worthy on fields with large number of unique values, possible unknown a priori, such as city or NAICS, then they should be applied. care should be taken because set of unique values can vary between years, and it might create problems when merging.
- validations:
  - codes are valid (i.e. can be found in lookup tables) for fields such as SIC, NAICS, FIPS, CBSA_CODE etc.
  - geo variable consistency: CBSA_LEVEL vs CBSA_CODE, lon-lat, nesting of areas
- few variables have many values like "00000", those should possibly be replaced with np.nan
  - subsidiary_number, parent_number, site_number, census_tract, csa_code, maybe others

In [ ]:
%nbdev_export

def convert_schema(datapackage_schema_path):
    """Convert old datapackage schema.json into a new file, to be used for data validation."""
    sch0 = json.load(open(datapackage_schema_path))

    def get_field_years(field_name):
        years = []
        for fl in sch0['field_lists']:
            if field_name in fl['fields']:
                years += fl['years']
        if 2015 in years:
            years += [2016, 2017]
        if field_name == 'gender':
            years += [2017]
        return sorted(years)


    sch = dict()
    sch['info'] = 'Schema for cleaned InfoGroup data.'
    sch['fields'] = list()

    for f0 in sch0['fields']:
        f = dict()
        name = f0['name']
        f['name'] = name.upper()
        f['years'] = get_field_years(name)
        if 'enum' in f0['constraints']:
            enum = f0['constraints']['enum'].copy()
            if '' in enum:
                enum.pop(enum.index(''))
            f['enum'] = enum
        if 'values' in f0:
            values = f0['values'].copy()
            if name == 'cbsa_level':
                del values['0'] # code never used
            f['enum_labels'] = values

        field_widths = {
            'ZIP': 5,
            'ZIP4': 4,
            'COUNTY_CODE': 3,
            'AREA_CODE': 3,
            'PHONE': 7,
            'SIC': 6,
            'SIC0': 6,
            'SIC1': 6,
            'SIC2': 6,
            'SIC3': 6,
            'SIC4': 6,
            'NAICS': 8,
            'YP_CODE': 5,
            'ABI': 9,
            'SUBSIDIARY_NUMBER': 9,
            'PARENT_NUMBER': 9,
            'SITE_NUMBER': 9,
            'CENSUS_TRACT': 6,
            'CENSUS_BLOCK': 1,
            'CBSA_CODE': 5,
            'CSA_CODE': 3,
            'FIPS_CODE': 5
        }
        if f['name'] in field_widths:
            f['width'] = field_widths[f['name']]

        f['original_name'] = f0['originalName']
        f['original_description'] = f0['originalDescription']
        sch['fields'].append(f)

    json.dump(sch, open(resources.get('infogroup/schema').path, 'w'), indent=1)
    

def get_schema(field_name=None, year=None):
    """Flexible function to get dataset schema.
    
    With no parameters, return full list of field dictionaries.
    With `year`, restrict to fields present in that year.
    With `field_name`, return dictionary for specific field (`year` is ignored in this case).
    """
    sch = json.load(resources.get(f'infogroup/schema').path.open())['fields']
    if field_name is not None:
        return [x for x in sch if x['name'] == field_name][0]
    if year is not None:
        return [x for x in sch if year in x['years']]
    return sch  

    
def pad_with_zeroes(df, schema_fields):
    """Prepend string column values with zeroes to have constant width."""

    for field in schema_fields:
        if 'width' in field:
            df[field['name']] = df[field['name']].str.zfill(field['width'])

def validate_raw_strings(df, schema_fields):
    """Validate values in raw InfoGroup data according to string constraints.
    Return list of dicts of invalid values.
    """
    
    constraints = {
        'ZIP': {'number': True},
        'ZIP4': {'number': True},
        'COUNTY_CODE': {'number': True},
        'AREA_CODE': {'number': True},
        'PHONE': {'number': True},
        'SIC': {'number': True},
        'SIC0': {'number': True},
        'SIC1': {'number': True},
        'SIC2': {'number': True},
        'SIC3': {'number': True},
        'SIC4': {'number': True},
        'NAICS': {'number': True},
        'YEAR': {'notna': True, 'number': True},
        'YP_CODE': {'number': True},
        'EMPLOYEES': {'number': True},
        'SALES': {'number': True},
        'PARENT_EMPLOYEES': {'number': True},
        'PARENT_SALES': {'number': True},
        'YEAR_EST': {'number': True},
        'ABI': {'unique': True, 'notna': True, 'number': True},
        'SUBSIDIARY_NUMBER': {'number': True},
        'PARENT_NUMBER': {'number': True},
        'SITE_NUMBER': {'number': True},
        'CENSUS_TRACT': {'number': True},
        'CENSUS_BLOCK': {'number': True},
        'LATITUDE': {'number': True},
        'LONGITUDE': {'number': True},
        'CBSA_CODE': {'number': True},
        'CSA_CODE': {'number': True},
        'FIPS_CODE': {'number': True}
    }
    
    # the above hard coded list of constraints must be consistent with field availability in given year
    constraints = {k: v for k, v in constraints.items() if k in df}
    
    
    for field in schema_fields:
        name = field['name']
        if 'enum' in field:
            if name not in constraints: constraints[name] = dict()
            constraints[name]['cats'] = field['enum']
        if 'width' in field:
            constraints[name]['nchar'] = field['width']
    
    return util.validate_values(df, constraints)


def convert_dtypes(df, schema_fields):
    """Inplace convert string columns to appropriate types."""
    
    for col in ['YEAR', 'EMPLOYEES', 'SALES', 'PARENT_EMPLOYEES', 'PARENT_SALES', 'YEAR_EST', 'LATITUDE', 'LONGITUDE']:
        df[col] = pd.to_numeric(df[col])
        
    for field in schema_fields:
        if 'enum' in field:
            df[field['name']] = pd.Categorical(df[field['name']], categories=field['enum'])
    

def validate_raw_numbers(df, year):
    """Validate values in raw InfoGroup data according to numerical constraints.
    Return list of dicts of invalid values.
    """
    
    constraints = {
        'YEAR': {'eq': year},
        'EMPLOYEES': {'ge': 0},
        'SALES': {'ge': 0},
        'PARENT_EMPLOYEES': {'ge': 0},
        'PARENT_SALES': {'ge': 0},
        'YEAR_EST': {'ge': 1000, 'le': year},
        'LATITUDE': {'ge': 0, 'le': 90},
        'LONGITUDE': {'ge': -180, 'le': 0}
    }
    return util.validate_values(df, constraints)

def replace_invalid(df, invalid_list, replacement=np.nan):
    """Replace invalid values."""
    for inv in invalid_list:
        df.loc[inv['idx'], inv['col']] = replacement
        logging.info(f'Replace invalid value `{inv["val"]}` with `{replacement}` at .loc[{inv["idx"]}, \'{inv["col"]}\']')

In [ ]:
convert_schema('/InfoGroup/data/original/raw/datapackage_schema.json')

In [ ]:
%nbdev_export
# Columns that do not appear in all years, as well as POPULATION_CODE that changes categories,
# are not includes in the full parquet dataset
shared_cols = [x['name']
               for x in json.load(resources.get('infogroup/schema').path.open())['fields']
               if (1997 in x['years']) and (x['name'] not in ['YEAR', 'POPULATION_CODE'])]

def load_validate_convert(year):
    """Run full processing pipeline on one year of data:
    load CVS, validate values, convert dtypes and save as parquet partition.
    """
    logging.info(f'Processing started for year {year}\n' + '-'*80)
    sch = json.load(resources.get(f'infogroup/schema').path.open())
    sch = [x for x in sch['fields'] if year in x['years']]
    
    # POPULATION_CODE has different values in 2016 and 2017
    if year >= 2016:
        for f in sch:
            if f['name'] == 'POPULATION_CODE':
                f['enum'] = list('0123456789')
                break

    df = pd.read_csv(resources.get(f'infogroup/orig/{year}').path, dtype='str', encoding='latin-1')

    df.rename(columns={x['original_name']: x['name'] for x in sch}, inplace=True)

    if year == 2009:
        pad_with_zeroes(df, sch)

    invalid_str = validate_raw_strings(df, sch)
    if len(invalid_str) < 100:
        replace_invalid(df, invalid_str)
    else:
        logging.error(f'Very many invalid_str values: {len(invalid_str)}, processing aborted')
        logging.error(invalid_str[:5])
        return

    convert_dtypes(df, sch)

    invalid_num = validate_raw_numbers(df, year)
    if len(invalid_num) < 100:
        replace_invalid(df, invalid_num)
    else:
        logging.error(f'Very many invalid_num values: {len(invalid_num)}, processing aborted')
        logging.error(invalid_num[:5])
        return
    
    df = df[shared_cols]
    partition_path = str(resources.get('infogroup/full').path / f'YEAR={year}')
    fastparquet.write(partition_path, df, file_scheme='hive', write_index=False, partition_on=['STATE'])

    logging.info(f'Processing finished for year {year}\n' + '-'*80)
    return partition_path


def build_parquet_dataset(n_cpus=1):
    """Create full parquet dataset from yearly CSV files."""
    
    logging.info('create_parquet_dataset() started.')
    
    p = resources.get('infogroup/full').path
    # Remove dataset files if they exist from before
    if p.exists():
        shutil.rmtree(p)
    p.mkdir()
    if n_cpus > 1:
        with mp.Pool(n_cpus) as pool:
            partition_paths = pool.map(load_validate_convert, data_years)
    else:
        partition_paths = [load_validate_convert(y) for y in data_years]
    _ = fastparquet.writer.merge(partition_paths)
    logging.info('create_parquet_dataset() finished.')

In [ ]:
%nbdev_export
def get_df(years=None, cols=None, states=None):
    """Return one year of InfoGroup data with appropriate data types.
    Subset of columns can be loaded by passing list to `cols`.
    """
    filters = []
    if years is not None:
        filters.append(('YEAR', 'in', years))
    if states is not None:
        filters.append(('STATE', 'in', states))
    res = resources.get('infogroup/full')
    df = pd.read_parquet(res.path, 'fastparquet', columns=cols, filters=filters)
    df['YEAR'] = df['YEAR'].astype(int)
    return df

# Descriptive summary statistics

Missing counts are reported as captured by `isna()` method. This will include empty strings from raw CSV for string dtype.

Some variables also have a special value such as "000" which is reported separately. These special values are not documented, and so are discovered by inspection.

In [ ]:
%nbdev_export

@memory.cache
def describe_variable(col, df=None, distribution=False, count_values=[]):
    """Return summary statistics of `col` for all years of data.

    Dataframe is read from disk unless given in `df`.
    Distribution moments are reported by setting `distribution` to True.
    Counts of values specified in `count_values` are reported.
    """
    
    if df is None:
        df = get_df(cols=['YEAR', col])
        
    stats = {}
    stats['Total'] = df.groupby('YEAR').size()
    stats['Total']['All'] = len(df)
    df['__FLAG'] = df[col].notna()
    stats['Not missing'] = df.groupby('YEAR')['__FLAG'].sum().astype(int)
    stats['Not missing']['All'] = stats['Not missing'].sum()
    stats['Missing'] = stats['Total'] - stats['Not missing']
    stats['Unique'] = df.groupby('YEAR')[col].nunique()
    stats['Unique']['All'] = df[col].nunique()

    if distribution:
        stats['Min'] = df.groupby('YEAR')[col].min()
        stats['Min']['All'] = stats['Min'].min()
        stats['Max'] = df.groupby('YEAR')[col].max()
        stats['Max']['All'] = stats['Max'].max()
        stats['Mean'] = df.groupby('YEAR')[col].mean()
        stats['Mean']['All'] = df[col].mean()
        stats['s.d.'] = df.groupby('YEAR')[col].std()
        stats['s.d.']['All'] = df[col].std()
        q = df.groupby('YEAR')[col].quantile([0.25, 0.5, 0.75]).unstack()
        qa = df[col].quantile([0.25, 0.5, 0.75])
        stats['25%'] = q[0.25]
        stats['25%']['All'] = qa[0.25]
        stats['50% (median)'] = q[0.5]
        stats['50% (median)']['All'] = qa[0.5]
        stats['75%'] = q[0.75]
        stats['75%']['All'] = qa[0.75]

    for val in count_values:
        df['__FLAG'] = (df[col] == val)
        stats[f'Value "{val}"'] = df.groupby('YEAR')['__FLAG'].sum().astype(int)
        stats[f'Value "{val}"']['All'] = stats[f'Value "{val}"'].sum()

    return pd.concat(stats, 1)


def style(df):
    """Apply formatting style to summary stats dataframe."""

    f = {}
    int_cols = ['Total', 'Not missing', 'Missing', 'Unique']
    int_cols += [c for c in df if c.startswith('Value "')]
    for c in int_cols:
        f[c] = '{:,}'
    for c in ['Min', 'Max', 'Mean', 's.d.', '25%', '50% (median)', '75%']:
        f[c] = '{:,g}'

    return df.style.format(f)


**COMPANY**: Name of business - will have blanks

In [ ]:
%nbdev_collapse_output
style(describe_variable('COMPANY'))

**ADDRESS**: Historical address

In [ ]:
%nbdev_collapse_output
style(describe_variable('ADDRESS'))

**CITY**: Historical address city

In [ ]:
%nbdev_collapse_output
style(describe_variable('CITY'))

**STATE**: Historical address state

In [ ]:
%nbdev_collapse_output
style(describe_variable('STATE', count_values=get_schema('STATE')['enum']))

**ZIP**: Historical address zip code

In [ ]:
%nbdev_collapse_output
style(describe_variable('ZIP'))

**ZIP4**: Historical address zip code zip + 4

In [ ]:
%nbdev_collapse_output
style(describe_variable('ZIP4'))

**COUNTY_CODE**: County code based upon location address/zip4 (postal)

In [ ]:
%nbdev_collapse_output
style(describe_variable('COUNTY_CODE'))

**AREA_CODE**: Area code of business

In [ ]:
%nbdev_collapse_output
style(describe_variable('AREA_CODE'))

**ID_CODE**: The code that indentifies the yellow page listing is for a business or for an individual.  this field helps clients indentify if the record represents a professional indivisual versus a firm record.  
Value labels:  
1: Individual  
2: Firm  

In [ ]:
%nbdev_collapse_output
style(describe_variable('ID_CODE', count_values=get_schema('ID_CODE')['enum']))

**EMPLOYEES_CODE**: Code indicating range of employees at that location  
Value labels:  
A: 1-4  
B: 5-9  
C: 10-19  
D: 20-49  
E: 50-99  
F: 100-249  
G: 250-499  
H: 500-999  
I: 1000-4999  
J: 5000-9999  
K: 10000-  

In [ ]:
%nbdev_collapse_output
style(describe_variable('EMPLOYEES_CODE', count_values=get_schema('EMPLOYEES_CODE')['enum']))

**SALES_CODE**: Corporate sales volume code (ranges) represents the total sales company wide  
Value labels:  
A: 1-499  
B: 500-999  
C: 1000-2499  
D: 2500-4999  
E: 5000-9999  
F: 10000-19999  
G: 20000-49999  
H: 50000-99999  
I: 100000-499999  
J: 500000-999999  
K: 1000000+  

In [ ]:
%nbdev_collapse_output
style(describe_variable('SALES_CODE', count_values=get_schema('SALES_CODE')['enum']))

**SIC**: This field contains the 6-digit sic code for the business’s primary activity

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC'))

**SIC_DESC**: The desciption for the sic code

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC_DESC'))

**NAICS**: The desciption for the primary naics code

In [ ]:
%nbdev_collapse_output
style(describe_variable('NAICS'))

**NAICS_DESC**: The desciption for the naics code

In [ ]:
%nbdev_collapse_output
style(describe_variable('NAICS_DESC'))

**SIC0**: A line of business that company engages in

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC0'))

**SIC0_DESC**: The desciption for the sic code

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC0_DESC'))

**SIC1**: This field identifies an additional activity of the business.  if there is no additional activity, this field will be blank.

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC1'))

**SIC1_DESC**: The desciption for the secondary sic code

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC1_DESC'))

**SIC2**: This field identifies an additional activity of the business.  if there is no additional activity, this field will be blank.

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC2'))

**SIC2_DESC**: The desciption for the secondary sic code

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC2_DESC'))

**SIC3**: This field identifies an additional activity of the business.  if there is no additional activity, this field will be blank.

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC3'))

**SIC3_DESC**: The desciption for the secondary sic code

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC3_DESC'))

**SIC4**: This field identifies an additional activity of the business.  if there is no additional activity, this field will be blank.

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC4'))

**SIC4_DESC**: The desciption for the secondary sic code

In [ ]:
%nbdev_collapse_output
style(describe_variable('SIC4_DESC'))

**YEAR**: Year of data

In [ ]:
%nbdev_collapse_output
style(describe_variable('YEAR'))

**YP_CODE**: A numeric value assigned to yellow page heading for the sic

In [ ]:
%nbdev_collapse_output
style(describe_variable('YP_CODE'))

**EMPLOYEES**: Number of employees at that location, could be modeled

In [ ]:
%nbdev_collapse_output
style(describe_variable('EMPLOYEES', distribution=True, count_values=[1]))

**SALES**: Sales volume at that location (in thousands)

In [ ]:
%nbdev_collapse_output
style(describe_variable('SALES', distribution=True))

**BUSINESS_STATUS**: Indicates if record is hq, sub, or branch  
Value labels:  
1: headquarters  
2: branch  
3: subsidiary  
9: standalone  

In [ ]:
%nbdev_collapse_output
style(describe_variable('BUSINESS_STATUS', count_values=get_schema('BUSINESS_STATUS')['enum']))

**IND_BYTE**: Contains "number of" info.  (# beds for nursing homes, # rooms for hotels)

In [ ]:
%nbdev_collapse_output
style(describe_variable('IND_BYTE'))

**YEAR_EST**: Year the business began operating

In [ ]:
%nbdev_collapse_output
style(describe_variable('YEAR_EST'))

**OFFICE_SIZE_CODE**: Number of professionals per office with the same phone number  
Value labels:  
A: 1  
B: 2  
C: 3  
D: 4  
E: 5-9  
F: 10+  

In [ ]:
%nbdev_collapse_output
style(describe_variable('OFFICE_SIZE_CODE', count_values=get_schema('OFFICE_SIZE_CODE')['enum']))

**HOLDING_STATUS**: Indicates if company is a public company, private company, or a branch

In [ ]:
%nbdev_collapse_output
style(describe_variable('HOLDING_STATUS', count_values=get_schema('HOLDING_STATUS')['enum']))

**ABI**: Also known as iusa number, abi number, infogroup number or location number, this provides a unique identifier for each business in the infogroup business database

In [ ]:
%nbdev_collapse_output
style(describe_variable('ABI'))

**SUBSIDIARY_NUMBER**: The subsidiary parent number identifies the business as a regional or subsidiary headquarters for a corporate family. the subsidiary will always have a parent and may or may not have branches assigned to it.

In [ ]:
%nbdev_collapse_output
style(describe_variable('SUBSIDIARY_NUMBER'))

**PARENT_NUMBER**: The parent number identifies the corporate parent of the business and also serves as the abi number for the headquarters site of the parent.  since all location of a business have the same ultimate parent number, this field provides ‘corporate ownwership’ linkage infomration.  this iinformation is not collected or maintained for the types organiation for wich ownership is ambiguous.  churches and schools, in particular, are not linked in the file for this reason

In [ ]:
%nbdev_collapse_output
style(describe_variable('PARENT_NUMBER'))

**PARENT_EMPLOYEES**: Parent actual employee size refers to the parent abi record only

In [ ]:
%nbdev_collapse_output
style(describe_variable('PARENT_EMPLOYEES', distribution=True))

**PARENT_SALES**: Parent actual sales refers to the parent abi record only

In [ ]:
%nbdev_collapse_output
style(describe_variable('PARENT_SALES', distribution=True))

**PARENT_EMPLOYEES_CODE**: Parent employee code refers to the parent abi record only  
Value labels:  
A: 1-4  
B: 5-9  
C: 10-19  
D: 20-49  
E: 50-99  
F: 100-249  
G: 250-499  
H: 500-999  
I: 1000-4999  
J: 5000-9999  
K: 10000-  

In [ ]:
%nbdev_collapse_output
style(describe_variable('PARENT_EMPLOYEES_CODE', count_values=get_schema('PARENT_EMPLOYEES_CODE')['enum']))

**PARENT_SALES_CODE**: Parent sales code refers to the parent abi records only  
Value labels:  
A: 1-499  
B: 500-999  
C: 1000-2499  
D: 2500-4999  
E: 5000-9999  
F: 10000-19999  
G: 20000-49999  
H: 50000-99999  
I: 100000-499999  
J: 500000-999999  
K: 1000000+

In [ ]:
%nbdev_collapse_output
style(describe_variable('PARENT_SALES_CODE', count_values=get_schema('PARENT_SALES_CODE')['enum']))

**SITE_NUMBER**: Designates related business at one site, identifying the primary business.  if abi# and site# are the same, then the record is primary business at the site.  if abi# and site# are different, then the record is a secondary business at the site.  determined through relationships between multiple data elements.

In [ ]:
%nbdev_collapse_output
style(describe_variable('SITE_NUMBER'))

**ADDRESS_TYPE**: Indicates if type of address  
Value labels:  
F: Firm  
G: General delivery  
H: High-rise  
M: Military  
P: Post office box  
R: Rural route or hwy contract  
S: Street  
N: Unknown  
: No match to Zip4

In [ ]:
%nbdev_collapse_output
style(describe_variable('ADDRESS_TYPE', count_values=get_schema('ADDRESS_TYPE')['enum']))

**CENSUS_TRACT**: Identifies a small geographic area for the purpose of collecting and compiling population and housing data.  census tracts are unique only within census county, and census counties are unique only within census state.

Special value "000000" - missing?

In [ ]:
%nbdev_collapse_output
style(describe_variable('CENSUS_TRACT', count_values=['000000']))

**CENSUS_BLOCK**: Bgs are subdivisions of census tracts and unique only within a specific census tract.  census tracts/block groups are assigned to address records via a geocoding process.

Special value "0" - missing?

In [ ]:
%nbdev_collapse_output
style(describe_variable('CENSUS_BLOCK', count_values=['0']))

**LATITUDE**: Parcel level assigned via point geo coding.  half of a pair of coordinates (the other being longitude)  provided in a formatted value, with decimals or a negative sign. not available in puerto rico & virgin island.

In [ ]:
%nbdev_collapse_output
style(describe_variable('LATITUDE', distribution=True))

**LONGITUDE**: Parcel level assigned via point geo coding.  note: longitudes are negatives values in the western hemisphere.  provided in its formatted value, with decimals or a negative sign. not available in puerto rico & virigin island

In [ ]:
%nbdev_collapse_output
style(describe_variable('LONGITUDE', distribution=True))

**MATCH_CODE**: Parcel level match code of the business location.  
Value labels:  
0: Site level  
2: Zip+2 centroid  
4: Zip+4 centroid  
P: Parcel  
X: Zip centroid

In [ ]:
%nbdev_collapse_output
style(describe_variable('MATCH_CODE', count_values=get_schema('MATCH_CODE')['enum']))

**CBSA_CODE**: Core bases statistical area (expanded msa code)

Special value "00000"

In [ ]:
%nbdev_collapse_output
style(describe_variable('CBSA_CODE', count_values=['00000']))

**CBSA_LEVEL**: Indicates if an area is a micropolitan or metropolitan area  
Value labels:  
1: Micropolitan  
2: Metropolitan

In [ ]:
%nbdev_collapse_output
style(describe_variable('CBSA_LEVEL', count_values=get_schema('CBSA_LEVEL')['enum']))

**CSA_CODE**: Adjoining cbsa's.  combination of metro and micro areas

Special value "000".

In [ ]:
%nbdev_collapse_output
style(describe_variable('CSA_CODE', count_values=['000']))

**FIPS_CODE**: First 2 bytes = state code, last 3 bytes = county code (location)

In [ ]:
%nbdev_collapse_output
style(describe_variable('FIPS_CODE'))

## Nonregular variables

The following variables are only available in select years.

- ...


Population code exists for all years, but coding changes after 2015.

**POPULATION_CODE**: The code for the resident population of the city in which the business is located, according to rand mcnally. some assignments can vary within a ciyt, such as when cities cross county lines. to maintain this granularity, the actual assignment is done at a zip level.

# Validation of geographic variables

Geographic variables

- ADDRESS: historical address
- CITY: historical address city
- STATE: historical address state
- ZIP: historical address zip code
- ZIP4: historical address zip code zip + 4
- COUNTY_CODE: county code based upon location address/zip4 (postal)
- AREA_CODE: area code of business
- ADDRESS_TYPE: indicates if type of address. "F": "Firm", "G": "General delivery", "H": "High-rise", "M": "Military", "P": "Post office box", "R": "Rural route or hwy contract", "S": "Street", "N": "Unknown", "": "No match to Zip4".
- CENSUS_TRACT: identifies a small geographic area for the purpose of collecting and compiling population and housing data.  census tracts are unique only within census county, and census counties are unique only within census state.  
- CENSUS_BLOCK: bgs are subdivisions of census tracts and unique only within a specific census tract.  census tracts/block groups are assigned to address records via a geocoding process.
- LATITUDE: parcel level assigned via point geo coding.  half of a pair of coordinates (the other being longitude)  provided in a formatted value, with decimals or a negative sign. not available in puerto rico & virgin island.
- LONGITUDE: parcel level assigned via point geo coding.  note: longitudes are negatives values in the western hemisphere.  provided in its formatted value, with decimals or a negative sign. not available in puerto rico & virigin island
- MATCH_CODE: parcel level match code of the business location. "0": "Site level", "2": "Zip+2 centroid", "4": "Zip+4 centroid", "P": "Parcel", "X": "Zip centroid".
- CBSA_CODE: core bases statistical area (expanded msa code)
- CBSA_LEVEL: indicates if an area is a micropolitan or metropolitan area. "1": "Micropolitan", "2": "Metropolitan"
- CSA_CODE: adjoining cbsa's.  combination of metro and micro areas
- FIPS_CODE: first 2 bytes = state code, last 3 bytes = county code (location)

In [ ]:
geo_cols = ['ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4', 'COUNTY_CODE', 'AREA_CODE', 'ADDRESS_TYPE', 
            'CENSUS_TRACT', 'CENSUS_BLOCK', 'LATITUDE', 'LONGITUDE', 'MATCH_CODE', 
            'CBSA_CODE', 'CBSA_LEVEL', 'CSA_CODE', 'FIPS_CODE']
total_count = {}
isna_count = {}
other_count = {}

for year in range(1997, 2018):
    df = get_df(year, cols=geo_cols)

    total_count[year] = len(df)

    isna_count[year] = {}
    for col in geo_cols:
        isna_count[year][col] = df[col].isna().sum()

    other_count[year] = {}
    other_count[year]['ADDRESS_TYPE'] = (df['ADDRESS_TYPE'] == 'N').sum()
    other_count[year]['CENSUS_TRACT'] = (df['CENSUS_TRACT'] == '000000').sum() # is 000000 a valid tract id?
    other_count[year]['CENSUS_BLOCK'] = (df['CENSUS_BLOCK'] == '0').sum() # is 0 a valid block id?
    other_count[year]['CBSA_CODE'] = (df['CBSA_CODE'] == '00000').sum()
    other_count[year]['CSA_CODE'] = (df['CSA_CODE'] == '000').sum()

## STATE, COUNTY_CODE and FIPS_CODE

- STATE is never missing
- Tiny fraction (0.0001%) have missing COUNTY_CODE or FIPS_CODE
- Until 2012, about 2% have inconsistent codes, and only a few after that

In [ ]:
state_fips_map = {
'AL':'01',
'AK':'02',
'AS':'60',
'AZ':'04',
'AR':'05',
'CA':'06',
'CO':'08',
'CT':'09',
'DE':'10',
'DC':'11',
'FL':'12',
'FM':'64',
'GA':'13',
'GU':'66',
'HI':'15',
'ID':'16',
'IL':'17',
'IN':'18',
'IA':'19',
'KS':'20',
'KY':'21',
'LA':'22',
'ME':'23',
'MH':'68',
'MD':'24',
'MA':'25',
'MI':'26',
'MN':'27',
'MS':'28',
'MO':'29',
'MT':'30',
'NE':'31',
'NV':'32',
'NH':'33',
'NJ':'34',
'NM':'35',
'NY':'36',
'NC':'37',
'ND':'38',
'MP':'69',
'OH':'39',
'OK':'40',
'OR':'41',
'PW':'70',
'PA':'42',
'PR':'72',
'RI':'44',
'SC':'45',
'SD':'46',
'TN':'47',
'TX':'48',
'UM':'74',
'UT':'49',
'VT':'50',
'VA':'51',
'VI':'78',
'WA':'53',
'WV':'54',
'WI':'55',
'WY':'56'
}

In [ ]:
logging.basicConfig(filename=resources.paths['root']/'tmp/geo_valid.log', level=logging.INFO, format='%(message)s', force=True)
# logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(message)s', force=True)
crosstabs = {}
for year in range(1997, 2018):
    df = get_df(year, cols=['STATE', 'COUNTY_CODE', 'FIPS_CODE'])

    df['_STATE_CODE'] = df['STATE'].map(state_fips_map, 'ignore')
    state_isna = df['STATE'].isna()
    state_isna.name = 'STATE is NA'
    county_isna = df['COUNTY_CODE'].isna()
    county_isna.name = 'COUNTY_CODE is NA'
    fips_isna = df.FIPS_CODE.isna()
    fips_isna.name = 'FIPS_CODE is NA'
    county_eq_fips = (df['COUNTY_CODE'] == df['FIPS_CODE'].str[2:])
    county_eq_fips.name = 'COUNTY_CODE consistent with FIPS_CODE'
    state_eq_fips = (df['_STATE_CODE'] == df['FIPS_CODE'].str[:2])
    state_eq_fips.name = 'STATE consistent with FIPS_CODE'

    crosstab = pd.crosstab([state_isna, county_isna, fips_isna], [county_eq_fips, state_eq_fips])
    crosstabs[year] = crosstab.stack([0, 1])
    logging.info(f'''
    ---- {year} ----
    STATE, COUNTY_CODE and FIPS_CODE consistency

    {crosstab}
    ''')

In [ ]:
ct = pd.concat(crosstabs).unstack(0).fillna(0).astype(int).T
ct = ct.loc[:, ~(ct == 0).all()].droplevel(0, 1)
ct

In [ ]:
ctf = pd.DataFrame()
ctf['Valid'] = ct.loc[:, (False, False, True, True)]
ctf['Missing COUNTY_CODE or FIPS_CODE'] = ct[True].sum(1) + ct[(False, True,)].iloc[:, 0]
ctf['Inconsistent codes'] = ct.sum(1) - ctf.sum(1)
pd.testing.assert_series_equal(ct.sum(1), ctf.sum(1))
ctf.loc['Total', :] = ctf.sum()
ctf['Total'] = ctf.sum(1)
ctf = ctf.astype(int)
ctf.style.format('{:,}')

In [ ]:
ctfr = ctf.copy()
for c in ctfr:
    ctfr[c] /= ctfr['Total']
ctfr.style.format('{:.3%}')

## CBSA_CODE and CBSA_LEVEL

# Tests